In [4]:
ENV["PYTHONPATH"]="/home/gridsan/aligho/.local/lib/python3.8/site-packages/PyNormaliz-2.15-py3.8-linux-x86_64.egg";
using Crystalline, MPBUtils, JLD2, LinearAlgebra, StaticArrays, SymmetryBases, PyPlot, DelimitedFiles, PrettyTables, Brillouin
using PyCall
include("./symeigs/corner_charges.jl")
include("./symeigs/wyckoffs_dict.jl");
cm = 1/2.54;

In [5]:
atolv = [1e-2, 2e-2, 4e-2, 6e-2, 8e-2, 1e-1] # (1e-2 is the default used in our version of MPBUtils.jl)
revisions_dir = "./Revisions/Resolution_Sweep/"
symeigs_dir = "./symeigs/";  # Relative paths to relevant directories
sgs = [2, 10, 13, 16]; # Only examine p2, p4, p3, and p6

In [6]:
stable_count_atol_sweep = Dict{Tuple{Integer, Integer, Integer, String, Integer}, Vector{Float64}}() # Key is (sg, id_eps=3, resolution, polarization, atol)
fragile_count_atol_sweep = Dict{Tuple{Integer, Integer, Integer, String, Integer}, Vector{Float64}}()

cum_stable_count_atol_sweep = Dict{Tuple{Integer, Integer, Integer, String, Integer}, Vector{Float64}}()
cum_fragile_count_atol_sweep = Dict{Tuple{Integer, Integer, Integer, String, Integer}, Vector{Float64}}()

total_sample_count_atol_sweep = Dict{Tuple{Integer, Integer, Integer, String, Integer}, Vector{Float64}}()

# We divide by 100 since we have 10000 samples per relevant category, so we divide by 10000 and multiply by 100 (to get a percentage)

# Below we sweep through atols 2e-2, 4e-2, 6e-2, 8e-2, 1e-1
for sg in sgs
    loaded_data = load(revisions_dir*"sg$sg-data-atolsweep.jld2")
    
    stable_count_dict = loaded_data["stable_count"]
    trivial_count_dict = loaded_data["trivial_count"]
    fragile_count_dict = loaded_data["fragile_count"]
    
    cum_stable_count_dict = loaded_data["cum_stable_count"]
    cum_trivial_count_dict = loaded_data["cum_trivial_count"]
    cum_fragile_count_dict = loaded_data["cum_fragile_count"]

    merge!(total_sample_count_atol_sweep, Dict(x => (stable_count_dict[x] + trivial_count_dict[x] + fragile_count_dict[x])[1:3] for (x, _) in stable_count_dict     ))
    merge!(stable_count_atol_sweep,  Dict(x => y[1:3]./100 for (x, y) in stable_count_dict))
    merge!(fragile_count_atol_sweep, Dict(x => y[1:3]./100  for (x, y) in fragile_count_dict))
    merge!(cum_stable_count_atol_sweep,  Dict(x => y[1:3]./100  for (x, y) in cum_stable_count_dict))
    merge!(cum_fragile_count_atol_sweep, Dict(x => y[1:3]./100 for (x, y) in cum_fragile_count_dict))
end

#Now have to merge our dictionaries with the ones relevant for atol = 1e-2

for sg in sgs
    loaded_data = load(revisions_dir*"sg$sg-data.jld2")
    
    stable_count_dict = loaded_data["stable_count"]
    trivial_count_dict = loaded_data["trivial_count"]
    fragile_count_dict = loaded_data["fragile_count"]
    
    cum_stable_count_dict = loaded_data["cum_stable_count"]
    cum_trivial_count_dict = loaded_data["cum_trivial_count"]
    cum_fragile_count_dict = loaded_data["cum_fragile_count"]

    merge!(total_sample_count_atol_sweep , Dict((x[1], x[2], x[3], x[4], 0) => (stable_count_dict[x] + trivial_count_dict[x] + fragile_count_dict[x])[1:3] for (x, _) in stable_count_dict     ))
    merge!(stable_count_atol_sweep ,  Dict((x[1], x[2], x[3], x[4], 0) => y[1:3]./100 for (x, y) in stable_count_dict))
    merge!(fragile_count_atol_sweep , Dict((x[1], x[2], x[3], x[4], 0)=> y[1:3]./100  for (x, y) in fragile_count_dict))
    merge!(cum_stable_count_atol_sweep ,  Dict((x[1], x[2], x[3], x[4], 0) => y[1:3]./100  for (x, y) in cum_stable_count_dict))
    merge!(cum_fragile_count_atol_sweep , Dict((x[1], x[2], x[3], x[4], 0) => y[1:3]./100 for (x, y) in cum_fragile_count_dict))
end

for sg in sgs
    loaded_data = load(symeigs_dir*"sg$sg-data.jld2")
    
    stable_count_dict = loaded_data["stable_count"]
    trivial_count_dict = loaded_data["trivial_count"]
    fragile_count_dict = loaded_data["fragile_count"]
    
    cum_stable_count_dict = loaded_data["cum_stable_count"]
    cum_trivial_count_dict = loaded_data["cum_trivial_count"]
    cum_fragile_count_dict = loaded_data["cum_fragile_count"]

    merge!(total_sample_count_atol_sweep, Dict((x[1], x[2], 9, x[3], 0) => (stable_count_dict[x] + trivial_count_dict[x] + fragile_count_dict[x])[1:3] for (x, _) in stable_count_dict     ))
    merge!(stable_count_atol_sweep,  Dict((x[1], x[2], 9, x[3], 0) => y[1:3] ./ 100 for (x, y) in stable_count_dict))
    merge!(fragile_count_atol_sweep, Dict((x[1], x[2], 9, x[3], 0) => y[1:3] ./ 100 for (x, y) in fragile_count_dict))
    merge!(cum_stable_count_atol_sweep,  Dict((x[1], x[2], 9, x[3], 0) => y[1:3] ./ 100 for (x, y) in cum_stable_count_dict))
    merge!(cum_fragile_count_atol_sweep, Dict((x[1], x[2], 9, x[3], 0) => y[1:3] ./ 100 for (x, y) in cum_fragile_count_dict))
end


In [7]:
corner_data_atol_sweep = Dict{Tuple{Integer, Integer, Integer, String, Integer}, Vector{Float64}}()

for atol_idx in 1:5
    for mode in ["te", "tm"]
        for sg in [2, 10, 13, 16]
            loaded_data = load(revisions_dir*"/sg$sg-corner-data-atolsweep.jld2")
            cum_corner_charges = loaded_data["cum_corner_charges"]
            for res_idx in 6:9
                x = (sum([cum_corner_charges[(sg, 3, res_idx, mode, wp, atol_idx)] for wp in sg_wyckoffs_dict[sg]])/length(sg_wyckoffs_dict[sg]))[1:3] ./ 100
                merge!(corner_data_atol_sweep, Dict((sg, 3, res_idx, mode, atol_idx) => x) )
            end
        end
    end
end

for mode in ["te", "tm"]
    for sg in [2, 10, 13, 16]
        loaded_data = load(revisions_dir*"/sg$sg-corner-data.jld2")
        cum_corner_charges = loaded_data["cum_corner_charges"]
        for res_idx in 1:8
            x = (sum([cum_corner_charges[(sg, 3, res_idx, mode, wp)] for wp in sg_wyckoffs_dict[sg]])/length(sg_wyckoffs_dict[sg]))[1:3] ./ 100
            merge!(corner_data_atol_sweep, Dict((sg, 3, res_idx, mode, 0) => x) )
        end
    end
end

for mode in ["te", "tm"]
    for sg in [2, 10, 13, 16]
        loaded_data = load(symeigs_dir*"/sg$sg-corner-data.jld2")
        cum_corner_charges = loaded_data["cum_corner_charges"]
        x = (sum([cum_corner_charges[(sg, 3, mode, wp)] for wp in sg_wyckoffs_dict[sg]])/length(sg_wyckoffs_dict[sg]))[1:3] ./ 100
        merge!(corner_data_atol_sweep, Dict((sg, 3, 9, mode, 0) => x) )
    end
end


In [5]:
# First analyze fragile topology

In [8]:
diffs_dict = Dict{Tuple{String, Integer, String}, Vector{Matrix{Float64}}}()

for (stat_type, data) in zip(["stable", "fragile", "corner"], [stable_count_atol_sweep, fragile_count_atol_sweep, corner_data_atol_sweep])
    for sg in sgs
        for mode in ["te", "tm"]
            atol_res_matv = Vector{Matrix{Float64}}(undef, 3)
            for mult_idx in eachindex(atol_res_matv)
                diffs = zeros(6, 4)
                compare_against_key = (sg, 3, 9, mode, 0) # id_eps = 3, resolution of 64, atol = 1e-2
                compare_against = data[compare_against_key] 
                for atol_idx in 0:5
                    for res_idx in 6:9
                        diffs[atol_idx+1, res_idx-5] = round(data[(sg, 3, res_idx, mode, atol_idx)][mult_idx]  - compare_against[mult_idx], digits=2)
                    end
                end
                atol_res_matv[mult_idx] = abs.(diffs)
            end
            diffs_dict[(stat_type, sg, mode)] = atol_res_matv
        end
    end
end

In [10]:
# Corner charge statistical differences for p6, multiplet 3
mult_idx = 3
sgnum = 16
diffs_dict[("corner", sgnum, "te")][mult_idx][:, [1, 4, 2, 3]]

6×4 Matrix{Float64}:
 0.24  0.0   0.2   0.16
 0.17  0.1   0.27  0.19
 0.07  0.22  0.35  0.15
 0.06  0.32  0.33  0.16
 0.21  0.29  0.31  0.18
 0.19  0.38  0.28  0.17

In [15]:
# Stable statistical differences for p6, multiplet 3
mult_idx = 3
sgnum = 16
diffs_dict[("stable", sgnum, "te")][mult_idx][:, [1, 4, 2, 3]]

6×4 Matrix{Float64}:
 0.36  0.0   0.67  0.89
 0.04  0.53  0.88  0.9
 0.47  0.71  0.95  0.98
 0.63  0.68  0.98  1.04
 0.78  0.72  0.99  1.04
 0.81  0.76  0.98  1.06

In [16]:
# Fragile statistical differences for p6, multiplet 3
mult_idx = 3
sgnum = 16
diffs_dict[("fragile", sgnum, "te")][mult_idx][:, [1, 4, 2, 3]]

6×4 Matrix{Float64}:
 0.12  0.0   0.31  0.38
 0.06  0.19  0.33  0.39
 0.14  0.17  0.37  0.37
 0.21  0.25  0.39  0.41
 0.22  0.31  0.38  0.41
 0.25  0.33  0.39  0.44

In [13]:
# Corner charge statistical differences for p6, multiplet 2
mult_idx = 2
sgnum = 16
diffs_dict[("corner", sgnum, "te")][mult_idx][:, [1, 4, 2, 3]]

6×4 Matrix{Float64}:
 0.63  0.0   0.68  0.9
 0.2   0.37  0.9   0.98
 0.01  0.46  0.94  0.97
 0.13  0.7   0.99  1.03
 0.2   0.76  1.01  1.04
 0.25  0.8   1.04  1.07

In [17]:
# Stable charge statistical differences for p6, multiplet 2
mult_idx = 2
sgnum = 16
diffs_dict[("stable", sgnum, "te")][mult_idx][:, [1, 4, 2, 3]]

6×4 Matrix{Float64}:
 0.65  0.0   0.47  0.78
 0.13  0.27  0.55  0.79
 0.14  0.38  0.69  0.87
 0.37  0.49  0.72  0.92
 0.5   0.54  0.75  0.93
 0.54  0.52  0.79  0.91

In [18]:
# Fragile charge statistical differences for p6, multiplet 2
mult_idx = 2
sgnum = 16
diffs_dict[("fragile", sgnum, "te")][mult_idx][:, [1, 4, 2, 3]]

6×4 Matrix{Float64}:
 0.07  0.0   0.11  0.06
 0.07  0.05  0.13  0.09
 0.02  0.09  0.11  0.08
 0.01  0.12  0.08  0.09
 0.05  0.14  0.07  0.08
 0.04  0.14  0.07  0.08

In [14]:
# Corner charge statistical differences for p6, multiplet 1
mult_idx = 1
sgnum = 16
diffs_dict[("corner", sgnum, "te")][mult_idx][:, [1, 4, 2, 3]]

6×4 Matrix{Float64}:
 0.16  0.0   0.25  0.28
 0.03  0.13  0.24  0.29
 0.11  0.15  0.31  0.32
 0.18  0.21  0.31  0.31
 0.24  0.25  0.33  0.31
 0.28  0.25  0.33  0.33

In [19]:
# Corner charge statistical differences for p6, multiplet 1
mult_idx = 1
sgnum = 16
diffs_dict[("stable", sgnum, "te")][mult_idx][:, [1, 4, 2, 3]]

6×4 Matrix{Float64}:
 0.06  0.0   0.14  0.2
 0.14  0.11  0.14  0.22
 0.2   0.23  0.21  0.27
 0.3   0.27  0.26  0.28
 0.37  0.33  0.28  0.28
 0.36  0.33  0.3   0.29

In [20]:
# Corner charge statistical differences for p6, multiplet 1
mult_idx = 1
sgnum = 16
diffs_dict[("fragile", sgnum, "te")][mult_idx][:, [1, 4, 2, 3]]

6×4 Matrix{Float64}:
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0

In [21]:
mult_idx = 1
sgnum = 13
diffs_dict[("corner", sgnum, "te")][mult_idx][:, [1, 4, 2, 3]]

6×4 Matrix{Float64}:
 0.15  0.0   0.27  0.34
 0.02  0.14  0.32  0.35
 0.09  0.23  0.34  0.36
 0.16  0.27  0.35  0.37
 0.21  0.29  0.36  0.38
 0.23  0.31  0.36  0.38

In [24]:
mult_idx = 2
sgnum = 13
diffs_dict[("corner", sgnum, "te")][mult_idx][:, [1, 4, 2, 3]]

6×4 Matrix{Float64}:
 0.32  0.0   0.44  0.55
 0.07  0.2   0.5   0.6
 0.05  0.38  0.57  0.65
 0.16  0.45  0.59  0.67
 0.25  0.49  0.61  0.68
 0.28  0.54  0.61  0.68

In [25]:
mult_idx = 3
sgnum = 13
diffs_dict[("corner", sgnum, "te")][mult_idx][:, [1, 4, 2, 3]]

6×4 Matrix{Float64}:
 0.12  0.0   0.17  0.2
 0.02  0.09  0.2   0.24
 0.11  0.22  0.25  0.28
 0.16  0.28  0.28  0.3
 0.21  0.31  0.29  0.3
 0.23  0.32  0.31  0.3

In [26]:
# Corner charge statistical differences for p6, multiplet 1
mult_idx = 1
sgnum = 16
diffs_dict[("fragile", sgnum, "te")][mult_idx][:, [1, 4, 2, 3]]

6×4 Matrix{Float64}:
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0

In [30]:
# Corner charge statistical differences for p6, multiplet 1
mult_idx = 2
sgnum = 13
diffs_dict[("fragile", sgnum, "te")][mult_idx][:, [1, 4, 2, 3]]

6×4 Matrix{Float64}:
 0.12  0.0   0.14  0.18
 0.05  0.08  0.18  0.18
 0.05  0.14  0.19  0.2
 0.07  0.17  0.2   0.21
 0.09  0.17  0.2   0.22
 0.12  0.18  0.2   0.22

In [29]:
# Corner charge statistical differences for p6, multiplet 1
mult_idx = 3
sgnum = 13
diffs_dict[("fragile", sgnum, "te")][mult_idx][:, [1, 4, 2, 3]]

6×4 Matrix{Float64}:
 0.92  0.0   1.18  1.49
 0.25  0.5   1.35  1.55
 0.3   0.92  1.48  1.64
 0.59  1.1   1.57  1.71
 0.79  1.14  1.58  1.74
 0.89  1.21  1.6   1.77